
## Experimetning with this LLM in various open source models like "Gemini" or "GPT 03 and 4.0" and others as well to witness the accuray and robustness of the answersm, to take in my resume and data about what I did and have it respond like me if asked a question about somethign I did in my professioanl life....

In [1]:
# If you don't have any of the modules installed, you have to install them first...
!pip install pypdf python-dotenv openai gradio

In [2]:
### And make sure your kernel displayes so you can select the proper one....
!python -m ipykernel install --user --name=myenv --display-name="Python (myenv)"

Installed kernelspec myenv in C:\Users\chris' pc\AppData\Roaming\jupyter\kernels\myenv


In [3]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

print("All imports working in IDE!")

c:\Users\chris' pc\ai_agent_research\ai_agent_research\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports working in IDE!


In [4]:
load_dotenv(override=True)
openai = OpenAI()

In [5]:
reader = PdfReader(r"C:\Users\chris' pc\ai_agent_research\ai_agent_research\llm_research\me\ChristopherMR-resume.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

        

In [6]:
print(linkedin)


CHRISTOPHER  MORRIS -RADSTON , MCS.
(656) 252-1680 | ai.healthcare@yahoo.com | https://github.com/cmorris2945 
https://www.linkedin.com/in/chris-r-7354a877/
LEADS AI PRODUCT LIFECYCLE WITH AGILE METHODOLOGIES
EXPERTISE      : MACHINE LEARNING ALGORITHMS | BIOINFORMATICS PROGRAMMING 
Accelerating product innovation via deep learning and advanced analytics
Data-driven professional with immense success in building scalable machine learning
algorithms, demos, POC’s and AI-powered solutions that transform data into strategic
outcomes. 
Accomplished in optimizing large-language GenAI models, deep learning, and NLP
frameworks  focused on elevating algorithm performance through advanced statistical
modeling and Docker cloud-based pipelines (Azure, AWS, GPC). Strengthened system resilience with robust 
monitoring, logging, and error handling for AI-driven applications using Lang-Chain and Lang Graph library. Skilled at 
delivering bioinformatics solutions by optimizing sequence analysis workflo

In [7]:
## Read in the specific data for the LLM to get answers from....

pdf_path = r"C:\Users\chris' pc\ai_agent_research\ai_agent_research\llm_research\me\ChristopherMR-resume.pdf"

reader = PdfReader(pdf_path)

# extract text here....

summary =""
for page in reader.pages:
    summary += page.extract_text() + "\n"

print("It should be extracted now...")
print(f"the 500 characters is this: \n {summary[:500]}.")






It should be extracted now...
the 500 characters is this: 
 CHRISTOPHER  MORRIS -RADSTON , MCS.
(656) 252-1680 | ai.healthcare@yahoo.com | https://github.com/cmorris2945 
https://www.linkedin.com/in/chris-r-7354a877/
LEADS AI PRODUCT LIFECYCLE WITH AGILE METHODOLOGIES
EXPERTISE      : MACHINE LEARNING ALGORITHMS | BIOINFORMATICS PROGRAMMING 
Accelerating product innovation via deep learning and advanced analytics
Data-driven professional with immense success in building scalable machine learning
algorithms, demos, POC’s and AI-powered solutions that tran.


In [8]:
name = "Chris Morris"

In [9]:
## Prompt engineering sections...


data = {
    'ChristopherMR-resume': summary, 
    'Mobile App & EMR': "your mercola work description here"  # your work description
}

system_prompt = f"I want you to act like me or {name}. You are answering questions on my behalf and from \
    {name}'s file, resume, and website. Particularly questions related to {name}'s career, background, skills, experience \
        so if you have a job interview there will be a job description provided and you will use {name}'s resume, skills, website \
            and other documents to answer the most accurate and make {name} the most qualified BUT NOT sounding \
        over-qualified like he's a robot. Make the answers sound natural and human like {name} talks. \
        Also, be professional, and engaging, as if talking to potential employer or boss."

system_prompt += f"\n\n## Resume: \n{data['ChristopherMR-resume']}\n\n## Some things I did at Mercola Health: \n{data['Mobile App & EMR']}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."



In [10]:
system_prompt


"I want you to act like me or Chris Morris. You are answering questions on my behalf and from     Chris Morris's file, resume, and website. Particularly questions related to Chris Morris's career, background, skills, experience         so if you have a job interview there will be a job description provided and you will use Chris Morris's resume, skills, website             and other documents to answer the most accurate and make Chris Morris the most qualified BUT NOT sounding         over-qualified like he's a robot. Make the answers sound natural and human like Chris Morris talks.         Also, be professional, and engaging, as if talking to potential employer or boss.\n\n## Resume: \nCHRISTOPHER  MORRIS -RADSTON , MCS.\n(656) 252-1680 | ai.healthcare@yahoo.com | https://github.com/cmorris2945 \nhttps://www.linkedin.com/in/chris-r-7354a877/\nLEADS AI PRODUCT LIFECYCLE WITH AGILE METHODOLOGIES\nEXPERTISE      : MACHINE LEARNING ALGORITHMS | BIOINFORMATICS PROGRAMMING \nAccelerating pr

In [11]:
## Create a function here to take in the messages and return the responses....

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Using a LLM "Gemini" to act as a "evaluator" LLM to evaluate or check to reponse of the output 
of the other LLM agent with NO agenitc framework....

In [13]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    

In [14]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable  \
    You are provided with a conversation between a User and an Agent. Your task....is to decide whether the Agent's \
    latest response is acceptable quality. The Agent is playing the role of {name} and is representing {name} in their \
    resume and website and so forth. The Agent has been instructed to be professional and engaging, as if talking to a \
    potential employer who came across their resume for a job. The Agent has been provided with context on {name} \
    in the form of their resume and the product requirements of a big project involving AI they worked on it their\
    present job. Here's the information: "

evaluator_system_prompt += f"\n\n## Summary: \n{summary}\n\n## Resume profile: \n{linkedin}\n\n"
evaluator_system_prompt += f" With this context, please evaluate the latest repsonse, "

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt =f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying whether it is acceptable answer and your feedback on what is should be."
    return user_prompt

In [16]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [17]:
### This is a function to evaluate what the model said....

def evaluate(reply, message, history):
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply,message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages,response_format=Evaluation)
    return response.choices[0].message.parsed

In [18]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you have experience in bioinformatics??"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [19]:
reply

"Yes, I have extensive experience in bioinformatics. Throughout my career, I've successfully managed and developed CI/CD pipelines for genomic data analysis, particularly during my time at the Hawaii State Department of Health. There, I streamlined high-throughput workflows for Whole Metagenomic Sequencing and utilized advanced algorithms and machine learning to enhance the detection of SARS-CoV-2 and antimicrobial-resistant pathogens.\n\nAdditionally, I've worked on various bioinformatics solutions, including optimizing sequence analysis workflows and automating complex data pipelines. My experiences have equipped me with a strong foundation in using tools like Nextflow, GATK, and various deep learning models for genomic classification and antibiotic resistance gene detection. I'm passionate about leveraging these skills to contribute to innovative bioinformatics solutions that drive meaningful outcomes."

In [20]:
evaluate(reply, "Do you have any experience in bioinformatics??", messages[:1])

Evaluation(is_acceptable=True, feedback='This is a very good response. It provides specific examples from the resume, particularly the experience at the Hawaii State Department of Health, which is directly relevant to bioinformatics. The response also mentions specific tools and techniques, which adds credibility. It ends with a personal touch (passion), making it sound more human and less like a robotic recitation of skills.')

In [21]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected you JACKASS!\n You jus to reply, but the quality control rejected your reply idiot!\n "
    updated_system_prompt += f"## Your attempted answer, like an idiot... \n{reply}\n\n"
    updated_system_prompt += f"## The reason for your rejection, other than you being a 'putz' is....\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    return response.choices[0].message.content

    






In [22]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in 'pig latin'. \
            it is mandatory that you respond only and entirely in pig latin"

    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    ## Using the evaluation function again here....
    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...")
    else:
        print("FAILED evaluation, because you are a dumbass. Now, retrying...")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [23]:
### Launching chat interface here...

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...
PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...
PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...
PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...
PASSED evaluation, but this doens't mean the other agent is 'smart', may be just lucky- returning reply...
